In [1]:
import time
import torch
from torch import nn
import torchvision
import numpy as np
import torchvision.transforms as transforms
import torchvision.datasets as dset
from IPython.display import display, clear_output
from torch.autograd import Variable

In [2]:
def status(batch_size, ep, epoch, i, loss, acc, data_loader):
    # status
    clear_output(wait=True)
    print(str(ep) + '/' + str(epoch))
    print('batch: ' + str(i+1) + '/' + str(len(data_loader)) + 
             ' [' + '='*int((i+1)/(len(data_loader)/20)) +
              '>' + ' '*(20 - int((i+1)/(len(data_loader)/20))) +
              ']')
    print('Loss: %.4g   Accuracy: %.4g' % ((loss / ((i+1)*batch_size)),
                                            (acc / ((i+1)*batch_size))))

In [3]:
class PezzGen(nn.Module):
    def __init__(self):
        super().__init__()
        #self.dimIn = dimIn
        #self.dimOut = dimOut
        #self.filterNum = filterNum
        #actFun = nn.LeakyReLU(0.2, inplace=True)
        
        print("\n------Initializing PezzGen------\n")
        
        self.conv1 = nn.Conv2d(3, 12, kernel_size=3, stride=1,
                              padding=1)
        self.BNorm1 = nn.BatchNorm2d(12)
        self.actFun = nn.ReLU()
        self.pool1 = nn.MaxPool2d(2,2)
        
        self.bridge = nn.Conv2d(12, 36, kernel_size=5,stride=1,
                                padding=2)
        self.BNorm2 = nn.BatchNorm2d(36)
        
        self.inv1 = nn.ConvTranspose2d(36, 12, kernel_size=5,stride=2,
                                padding=2, output_padding=1)
        self.BNorm3 = nn.BatchNorm2d(12)
        self.actFun = nn.ReLU()
        self.up1 = nn.Conv2d(12, 3, kernel_size=3, stride=1,
                              padding=1)
        self.BNorm4 = nn.BatchNorm2d(3)
        self.tanh = nn.Tanh()
        
    def forward(self, x):
        x = self.pool1(self.actFun(self.BNorm1(self.conv1(x))))
        x = self.BNorm2(self.bridge(x))
        x = self.actFun(self.BNorm3(self.inv1(x)))
        x = self.BNorm4(self.up1(x))
        x = self.tanh(x)
        return x

net = PezzGen()
              


------Initializing PezzGen------



In [4]:
# parameters
batch_size = 50
img_size = 256
lr = 0.1
epoch = 100

# Generator
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
generator = nn.DataParallel(net)

img_dir = "./maps/"
trainset = dset.ImageFolder(root=img_dir,
                            transform = transforms.Compose([
                            transforms.Scale(size=img_size),
                            transforms.CenterCrop(size=(img_size,
                            img_size*2)),
                            transforms.ToTensor(),
                            ]))
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, 
                                          num_workers=2)

recon_loss_func = nn.MSELoss()
gen_optimizer = torch.optim.Adam(net.parameters(),lr=lr)




/anaconda3/lib/python3.7/site-packages/torchvision/transforms/transforms.py:207: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  warnings.warn("The use of the transforms.Scale transform is deprecated, " +


In [5]:
for ep in range(epoch):
    for i, (image, label) in enumerate(trainloader):
        
        satel_image, map_image = torch.chunk(image, chunks=2, dim=3)
        
        gen_optimizer.zero_grad()
        
        # print data
        #im = im[0,:,:,:]
        #print(image.shape)
        #plt.imshow(np.transpose(im, (1, 2, 0)))
        #plt.show()
        #im = satel_image[0,:,:,:]
        #plt.imshow(np.transpose(im, (1, 2, 0)))
        #plt.show()
        
        #time.sleep(2)
        
        x = Variable(satel_image)
        y_ = Variable(map_image)
        y = generator.forward(x)
        
        current_loss = recon_loss_func(y,y_)
        current_loss.backward()
        gen_optimizer.step()
        current_acc = np.array(1)
        
        # status
        status(batch_size, ep+1 , epoch, i, current_loss,
               current_acc, trainloader)
        

1/100
batch: 14/549 [>                    ]
Loss: 2.583e-05   Accuracy: 0.001429


Traceback (most recent call last):
Traceback (most recent call last):
  File "/anaconda3/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/anaconda3/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/anaconda3/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/anaconda3/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
  File "/anaconda3/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/anaconda3/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/anaconda3/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/anaconda3/lib/python3.7/multiprocessing/connection.py", 

KeyboardInterrupt: 